In [1]:
import geopandas as gpd
import geoai


In [2]:
satellite_imagery = './pabbi_satellite_image_COG.tif'
vector_file = './pabb_crop_V3.geojson'

In [3]:

gdf = gpd.read_file(vector_file)

print(f"Total Number of Parcels {len(gdf)}")

print(f"Number of Parcels with values (other than null): {gdf['Landuse_Ma'].notna().sum()}")

print(f"Details of Crop are : {gdf['Landuse_Ma'].value_counts()}")

Total Number of Parcels 7012
Number of Parcels with values (other than null): 6515
Details of Crop are : Landuse_Ma
Agriculture     5759
Stream           302
Other            229
Road/Streets     172
Graveyard         29
Built up          24
Name: count, dtype: int64


# Setting Class Ids for Land use Before generating Raster

In [4]:
label_map = {
    'Agriculture': 1,
    'Stream': 2,
    'Other': 3,
    'Road/Streets': 4,
    'Graveyard': 5,
    'Built up': 6
}

gdf['class_id'] = gdf['Landuse_Ma'].map(label_map)

gdf = gdf[gdf['class_id'].notna()]
gdf['class_id'] = gdf['class_id'].astype('uint8')
gdf=gdf.to_crs('EPSG:4326')
print(gdf.crs)

EPSG:4326


In [5]:
print(gdf['class_id'].unique())
print(gdf['class_id'].isnull().sum())
print(gdf['class_id'].dtype)


[1 6 2 4 5 3]
0
uint8


In [10]:
# Writing to temporary file
gdf.to_file('temp_cleaned_vector.geojson')

This code didnot work so i have to use the above code for manual vector to raster conversion.
geoai.vector_to_raster(
    vector_path=vector_file,
    reference_raster=satellite_imagery,
    attribute_field='class_id',
    output_path='Labelraster_For_Unet_v4.tif',
    dtype='uint8',
    plot_result=True,
    nodata=0
    )

In [8]:
geoai.view_raster('Labelraster_Manual.tif', cmap='tab20')

# Unet Part to create image tiles / patches

In [3]:
vector_path = 'temp_cleaned_vector.geojson'
labeled_raster_path = 'labelraster_Manual.tif'
out_folder = 'landcover_pabbi_satellite2'

In [4]:

tiles = geoai.export_geotiff_tiles(
    in_raster = satellite_imagery,
    out_folder= out_folder,
    in_class_data= labeled_raster_path,
    #class_value_field= 'labels',
    tile_size=256,
    stride=128,
    buffer_radius=0,
    skip_empty_tiles=True,
)

Detected in_class_data as raster: labelraster_Manual.tif
Raster CRS: EPSG:4326
Raster dimensions: 31447 x 23080

Raster info for ./pabbi_satellite_image_COG.tif:
  CRS: EPSG:4326
  Dimensions: 31447 x 23080
  Resolution: (2.6822089999998085e-06, 2.682208999999812e-06)
  Bands: 3
  Bounds: BoundingBox(left=71.731650084257, bottom=33.986464888201006, right=71.81599751067999, top=34.048370271921)
Found 6 unique classes in raster: [1 2 3 4 5 6]


Generated: 16753, With features: 16753: 100%|██████████| 44100/44100 [07:00<00:00, 104.80it/s]



------- Export Summary -------
Total tiles exported: 16753
Tiles with features: 16753 (100.0%)
Average feature pixels per tile: 117714.2
Output saved to: landcover_pabbi_satellite2

------- Georeference Verification -------


In [ ]:
# Train U-Net model
geoai.train_segmentation_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/unet_models",
    architecture="unet",
    encoder_name="resnet34",
    encoder_weights="imagenet",
    num_channels=4,
    num_classes=13,
    batch_size=8,
    num_epochs=50,
    learning_rate=0.001,
    val_split=0.2,
    verbose=True,
    checkpoint_path=f"{out_folder}/unet_models/checkpoint.pth",
    resume_training=True,
    plot_curves=True,
)

Using device: cuda
Found 16753 image files and 16753 label files
Training on 13402 images, validating on 3351 images
Starting training with unet + resnet34
Model parameters: 24,441,245
Epoch: 0, Batch: 0/1676, Loss: 2.6744, Time: 37.38s
Epoch: 0, Batch: 10/1676, Loss: 1.6480, Time: 2.06s
Epoch: 0, Batch: 20/1676, Loss: 1.2370, Time: 2.04s
Epoch: 0, Batch: 30/1676, Loss: 0.8287, Time: 2.06s
Epoch: 0, Batch: 40/1676, Loss: 0.5270, Time: 2.05s
Epoch: 0, Batch: 50/1676, Loss: 0.4519, Time: 2.05s
Epoch: 0, Batch: 60/1676, Loss: 1.6450, Time: 2.05s
Epoch: 0, Batch: 70/1676, Loss: 0.7865, Time: 2.05s
Epoch: 0, Batch: 80/1676, Loss: 1.1752, Time: 2.06s
Epoch: 0, Batch: 90/1676, Loss: 1.3902, Time: 2.06s
Epoch: 0, Batch: 100/1676, Loss: 0.9501, Time: 2.06s
Epoch: 0, Batch: 110/1676, Loss: 0.8420, Time: 2.05s
Epoch: 0, Batch: 120/1676, Loss: 0.5599, Time: 2.06s
Epoch: 0, Batch: 130/1676, Loss: 1.1044, Time: 2.06s
Epoch: 0, Batch: 140/1676, Loss: 1.2360, Time: 2.06s
Epoch: 0, Batch: 150/1676, Los

In [ ]:
# Define paths
masks_path = "naip_test_semantic_prediction.tif"
model_path = f"{out_folder}/unet_models/best_model.pth"

In [ ]:
# Run semantic segmentation inference
geoai.semantic_segmentation(
    input_path=test_raster_path,
    output_path=masks_path,
    model_path=model_path,
    architecture="unet",
    encoder_name="resnet34",
    num_channels=4,
    num_classes=13,
    window_size=512,
    overlap=256,
    batch_size=4,
)

In [7]:
geoai.write_colormap(masks_path, train_landcover_path, output=masks_path)

NameError: name 'masks_path' is not defined

In [ ]:
geoai.view_raster(masks_path, basemap=test_raster_url)